In [25]:
import pandas as pd
import numpy as np

In [119]:
train_df = pd.read_csv('datasets/titanic/train.csv')
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [120]:
train_df = train_df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'SibSp', 'Parch'])
train_df = pd.get_dummies(train_df, columns=['Sex']).drop(columns='Sex_female')
train_df = train_df.fillna({'Age': train_df['Age'].median()})
train_df.head()

,Survived,Pclass,Age,Sex_male
0,0,3,22.0,1
1,1,1,38.0,0
2,1,3,26.0,0
3,1,1,35.0,0
4,0,3,35.0,1


In [121]:
from data import normalize_columns
normalize_columns(train_df, columns=['Age'])
train_df.head()

,Survived,Pclass,Age,Sex_male
0,0,3,0.271174,1
1,1,1,0.472229,0
2,1,3,0.321438,0
3,1,1,0.434531,0
4,0,3,0.434531,1


In [122]:
X = train_df.drop(columns='Survived')
y = train_df['Survived']

In [148]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Самописный градиентный спуск + логистическая регрессия

In [150]:
from linear import StandardGradientDescent
from quality_functional import LogisticLoss
gd = StandardGradientDescent(alpha=0.01, S=1000, loss=LogisticLoss)
gd.fit(X_train, y_train.to_numpy())

In [151]:
y_pred = gd.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

In [152]:
from quality_functional import Accuracy
Accuracy.get_accuracy(y_pred, y_test)

0.8033898305084746

In [153]:
test_df = pd.read_csv('datasets/titanic/test.csv')
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [154]:
predicts_df = pd.DataFrame(test_df['PassengerId'])

In [155]:
test_df = test_df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'SibSp', 'Parch'])
test_df = pd.get_dummies(test_df, columns=['Sex']).drop(columns='Sex_female')
test_df = test_df.fillna({'Age': test_df['Age'].median()})
normalize_columns(test_df, columns=['Age'])
test_df.head()

,Pclass,Age,Sex_male
0,3,0.452723,1
1,3,0.617566,0
2,2,0.815377,1
3,3,0.353818,1
4,3,0.287881,0


In [156]:
predicts_df['Survived'] = (gd.predict(test_df) > 0.5).astype(int)

In [159]:
predicts_df.to_csv('predicts.csv', index=False)

### Результат ~0.77

# Логистическая регрессия sklearn

In [160]:
from sklearn.linear_model import LogisticRegression

In [162]:
lr = LogisticRegression(penalty='none')

In [163]:
lr.fit(X_train, y_train)

LogisticRegression(penalty='none')

In [166]:
Accuracy.get_accuracy(lr.predict(X_test), y_test)

0.8101694915254237

In [168]:
predicts_df['Survived'] = lr.predict(test_df)

In [170]:
predicts_df.to_csv('predicts.csv', index=False)

In [ ]:
## Результат 